# L2HMC for $U(1)$ Gauge Model 

### Imports

In [29]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from l2hmc_eager import dynamics_eager as _l2hmc
from l2hmc_eager import gauge_dynamics_eager as l2hmc
from l2hmc_eager.neural_nets import *
from utils.distributions import GMM, gen_ring
from utils.jacobian import _map, jacobian

%autoreload 2

In [30]:
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice, u1_plaq_exact

In [31]:
from l2hmc_eager.neural_nets import _custom_conv
from l2hmc_eager.neural_nets import _custom_dense

In [32]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [33]:
def train_one_iter(dynamics, x, optimizer,
                   loss_fn=l2hmc.compute_loss, global_step=None):
    loss, grads, out, accept_prob = l2hmc.loss_and_grads(
        dynamics, x, loss_fn=loss_fn
    )
    optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    return loss, out, accept_prob

In [34]:
def exact_plaquette_average(beta):
    return i1(beta) / i0(beta)

### Construct GaugeLattice with $U(1)$ gauge group

$$ S[\theta] = \beta \sum_{x;\,\, \nu\neq\mu} 1 - \cos(\theta_{\mu\nu})$$
with $\theta_{\mu\nu} \equiv \theta_{\mu}(x) + \theta_{\nu}(x +\hat \mu) - \theta_{\mu}(x + \hat \nu) - \theta_{\nu}(x)$ 

#### Heatbath Algorithm

In [14]:
time_size, space_size, dim, beta, num_samples = (16, 16, 2, 4., 5)
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples)
u1_samples = [sample.flatten() for sample in u1_lattice.samples]
u1_samples_tensor = tf.constant(np.stack(u1_samples), dtype=tf.float32)

In [187]:
eq_steps = 5000
acceptances = []
action_arr = [u1_lattice._total_action()]
avg_plaq_arr = [u1_lattice._average_plaquette()]
for i in range(eq_steps):
    action = u1_lattice._total_action()
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[-1]
    avg_plaq_arr.append(avg_plaq)
    action_arr.append(action)
    print(f"Step: {i:<5g}\t action: {action:<8.4g}\t "
          f"avg plaq: {avg_plaq:<8.4g}\t change: {change:<8.4g}")
    accept = 0
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            accept += u1_lattice._update_link(site, d)
    acceptances.append(accept)
# 12.2s for 500 equilibration steps

Step: 0    	 action: 0       	 avg plaq: 1       	 change: 0       
Step: 1    	 action: 19.82   	 avg plaq: 0.9226  	 change: -0.07743
Step: 2    	 action: 23.28   	 avg plaq: 0.9091  	 change: -0.0135 
Step: 3    	 action: 29.14   	 avg plaq: 0.8862  	 change: -0.02288
Step: 4    	 action: 31.69   	 avg plaq: 0.8762  	 change: -0.00999
Step: 5    	 action: 33.95   	 avg plaq: 0.8674  	 change: -0.008804
Step: 6    	 action: 37.17   	 avg plaq: 0.8548  	 change: -0.01258
Step: 7    	 action: 41.08   	 avg plaq: 0.8395  	 change: -0.01529
Step: 8    	 action: 36.98   	 avg plaq: 0.8555  	 change: 0.01602 
Step: 9    	 action: 36.7    	 avg plaq: 0.8566  	 change: 0.001085
Step: 10   	 action: 29.79   	 avg plaq: 0.8836  	 change: 0.02701 
Step: 11   	 action: 36.3    	 avg plaq: 0.8582  	 change: -0.02542
Step: 12   	 action: 35.51   	 avg plaq: 0.8613  	 change: 0.003061
Step: 13   	 action: 35.89   	 avg plaq: 0.8598  	 change: -0.001474
Step: 14   	 action: 37.31   	 avg plaq: 0.854

Step: 121  	 action: 30.22   	 avg plaq: 0.882   	 change: 0.01839 
Step: 122  	 action: 32.88   	 avg plaq: 0.8716  	 change: -0.01039
Step: 123  	 action: 31.05   	 avg plaq: 0.8787  	 change: 0.00716 
Step: 124  	 action: 34.54   	 avg plaq: 0.8651  	 change: -0.01366
Step: 125  	 action: 32.57   	 avg plaq: 0.8728  	 change: 0.007682
Step: 126  	 action: 39.18   	 avg plaq: 0.847   	 change: -0.0258 
Step: 127  	 action: 32.89   	 avg plaq: 0.8715  	 change: 0.02456 
Step: 128  	 action: 31.11   	 avg plaq: 0.8785  	 change: 0.00695 
Step: 129  	 action: 26.7    	 avg plaq: 0.8957  	 change: 0.01725 
Step: 130  	 action: 34.97   	 avg plaq: 0.8634  	 change: -0.03231
Step: 131  	 action: 36.21   	 avg plaq: 0.8586  	 change: -0.00486
Step: 132  	 action: 31.24   	 avg plaq: 0.878   	 change: 0.01943 
Step: 133  	 action: 27.97   	 avg plaq: 0.8908  	 change: 0.01277 
Step: 134  	 action: 33.59   	 avg plaq: 0.8688  	 change: -0.02197
Step: 135  	 action: 33.97   	 avg plaq: 0.8673 

Step: 242  	 action: 40.07   	 avg plaq: 0.8435  	 change: 0.002274
Step: 243  	 action: 34.81   	 avg plaq: 0.864   	 change: 0.02054 
Step: 244  	 action: 33.58   	 avg plaq: 0.8688  	 change: 0.004797
Step: 245  	 action: 34.86   	 avg plaq: 0.8638  	 change: -0.004986
Step: 246  	 action: 38.04   	 avg plaq: 0.8514  	 change: -0.01243
Step: 247  	 action: 38.46   	 avg plaq: 0.8497  	 change: -0.001665
Step: 248  	 action: 35.42   	 avg plaq: 0.8617  	 change: 0.01191 
Step: 249  	 action: 34.61   	 avg plaq: 0.8648  	 change: 0.003137
Step: 250  	 action: 34.83   	 avg plaq: 0.8639  	 change: -0.0008459
Step: 251  	 action: 43.85   	 avg plaq: 0.8287  	 change: -0.03522
Step: 252  	 action: 42.24   	 avg plaq: 0.835   	 change: 0.006287
Step: 253  	 action: 40.42   	 avg plaq: 0.8421  	 change: 0.007077
Step: 254  	 action: 39.8    	 avg plaq: 0.8445  	 change: 0.002454
Step: 255  	 action: 38.57   	 avg plaq: 0.8493  	 change: 0.004801
Step: 256  	 action: 39.06   	 avg plaq: 0.8

Step: 362  	 action: 36.4    	 avg plaq: 0.8578  	 change: -0.001153
Step: 363  	 action: 37.7    	 avg plaq: 0.8527  	 change: -0.00507
Step: 364  	 action: 39.23   	 avg plaq: 0.8468  	 change: -0.005981
Step: 365  	 action: 37.16   	 avg plaq: 0.8549  	 change: 0.008103
Step: 366  	 action: 38.59   	 avg plaq: 0.8493  	 change: -0.005588
Step: 367  	 action: 35.45   	 avg plaq: 0.8615  	 change: 0.01227 
Step: 368  	 action: 34.52   	 avg plaq: 0.8651  	 change: 0.003606
Step: 369  	 action: 32.73   	 avg plaq: 0.8721  	 change: 0.006998
Step: 370  	 action: 34.45   	 avg plaq: 0.8654  	 change: -0.006735
Step: 371  	 action: 32.47   	 avg plaq: 0.8732  	 change: 0.00775 
Step: 372  	 action: 32.95   	 avg plaq: 0.8713  	 change: -0.00186
Step: 373  	 action: 33.2    	 avg plaq: 0.8703  	 change: -0.0009722
Step: 374  	 action: 33.88   	 avg plaq: 0.8676  	 change: -0.002685
Step: 375  	 action: 36.79   	 avg plaq: 0.8563  	 change: -0.01135
Step: 376  	 action: 36.29   	 avg plaq: 

Step: 482  	 action: 32.69   	 avg plaq: 0.8723  	 change: 0.0133  
Step: 483  	 action: 34.1    	 avg plaq: 0.8668  	 change: -0.00554
Step: 484  	 action: 33.22   	 avg plaq: 0.8703  	 change: 0.003467
Step: 485  	 action: 31.02   	 avg plaq: 0.8788  	 change: 0.008584
Step: 486  	 action: 33.91   	 avg plaq: 0.8675  	 change: -0.01129
Step: 487  	 action: 34.74   	 avg plaq: 0.8643  	 change: -0.003251
Step: 488  	 action: 36.71   	 avg plaq: 0.8566  	 change: -0.007692
Step: 489  	 action: 31.66   	 avg plaq: 0.8763  	 change: 0.01971 
Step: 490  	 action: 33.57   	 avg plaq: 0.8689  	 change: -0.007443
Step: 491  	 action: 39.72   	 avg plaq: 0.8449  	 change: -0.02401
Step: 492  	 action: 37.07   	 avg plaq: 0.8552  	 change: 0.01033 
Step: 493  	 action: 37.54   	 avg plaq: 0.8534  	 change: -0.001831
Step: 494  	 action: 37.26   	 avg plaq: 0.8545  	 change: 0.001093
Step: 495  	 action: 37.56   	 avg plaq: 0.8533  	 change: -0.001157
Step: 496  	 action: 39.65   	 avg plaq: 0.

...

Step: 4453 	 action: 33.97   	 avg plaq: 0.8673  	 change: -0.008944
Step: 4454 	 action: 31.9    	 avg plaq: 0.8754  	 change: 0.008102
Step: 4455 	 action: 36.78   	 avg plaq: 0.8563  	 change: -0.01908
Step: 4456 	 action: 33.81   	 avg plaq: 0.8679  	 change: 0.01162 
Step: 4457 	 action: 33.78   	 avg plaq: 0.868   	 change: 9.096e-05
Step: 4458 	 action: 32.46   	 avg plaq: 0.8732  	 change: 0.005158
Step: 4459 	 action: 32.59   	 avg plaq: 0.8727  	 change: -0.0005133
Step: 4460 	 action: 29.81   	 avg plaq: 0.8836  	 change: 0.01089 
Step: 4461 	 action: 31.96   	 avg plaq: 0.8752  	 change: -0.008406
Step: 4462 	 action: 32.1    	 avg plaq: 0.8746  	 change: -0.0005602
Step: 4463 	 action: 33.08   	 avg plaq: 0.8708  	 change: -0.003834
Step: 4464 	 action: 31.3    	 avg plaq: 0.8777  	 change: 0.006975
Step: 4465 	 action: 32.24   	 avg plaq: 0.8741  	 change: -0.003689
Step: 4466 	 action: 34.61   	 avg plaq: 0.8648  	 change: -0.009244
Step: 4467 	 action: 33.21   	 avg pla

Step: 4573 	 action: 33.97   	 avg plaq: 0.8673  	 change: 0.001918
Step: 4574 	 action: 38.17   	 avg plaq: 0.8509  	 change: -0.0164 
Step: 4575 	 action: 32.31   	 avg plaq: 0.8738  	 change: 0.02287 
Step: 4576 	 action: 37.46   	 avg plaq: 0.8537  	 change: -0.02011
Step: 4577 	 action: 34.63   	 avg plaq: 0.8647  	 change: 0.01103 
Step: 4578 	 action: 35.08   	 avg plaq: 0.863   	 change: -0.001748
Step: 4579 	 action: 36.26   	 avg plaq: 0.8584  	 change: -0.004592
Step: 4580 	 action: 37.92   	 avg plaq: 0.8519  	 change: -0.006501
Step: 4581 	 action: 34.28   	 avg plaq: 0.8661  	 change: 0.01423 
Step: 4582 	 action: 33.42   	 avg plaq: 0.8695  	 change: 0.003366
Step: 4583 	 action: 37.82   	 avg plaq: 0.8523  	 change: -0.01719
Step: 4584 	 action: 43.71   	 avg plaq: 0.8293  	 change: -0.02302
Step: 4585 	 action: 41.64   	 avg plaq: 0.8373  	 change: 0.008076
Step: 4586 	 action: 33.68   	 avg plaq: 0.8684  	 change: 0.03111 
Step: 4587 	 action: 31.97   	 avg plaq: 0.87

Step: 4694 	 action: 31.53   	 avg plaq: 0.8768  	 change: -0.008818
Step: 4695 	 action: 31.84   	 avg plaq: 0.8756  	 change: -0.001209
Step: 4696 	 action: 33.83   	 avg plaq: 0.8679  	 change: -0.007771
Step: 4697 	 action: 33.99   	 avg plaq: 0.8672  	 change: -0.0006342
Step: 4698 	 action: 32.9    	 avg plaq: 0.8715  	 change: 0.004268
Step: 4699 	 action: 36.34   	 avg plaq: 0.858   	 change: -0.01346
Step: 4700 	 action: 34.96   	 avg plaq: 0.8634  	 change: 0.005412
Step: 4701 	 action: 36.53   	 avg plaq: 0.8573  	 change: -0.006159
Step: 4702 	 action: 32.74   	 avg plaq: 0.8721  	 change: 0.01483 
Step: 4703 	 action: 37.52   	 avg plaq: 0.8534  	 change: -0.01868
Step: 4704 	 action: 33.61   	 avg plaq: 0.8687  	 change: 0.01528 
Step: 4705 	 action: 37.38   	 avg plaq: 0.854   	 change: -0.01473
Step: 4706 	 action: 38.31   	 avg plaq: 0.8503  	 change: -0.003643
Step: 4707 	 action: 34.28   	 avg plaq: 0.8661  	 change: 0.01574 
Step: 4708 	 action: 36.55   	 avg plaq: 

Step: 4814 	 action: 38.33   	 avg plaq: 0.8503  	 change: 0.005687
Step: 4815 	 action: 41.03   	 avg plaq: 0.8397  	 change: -0.01053
Step: 4816 	 action: 40.9    	 avg plaq: 0.8402  	 change: 0.0004963
Step: 4817 	 action: 39.52   	 avg plaq: 0.8456  	 change: 0.005387
Step: 4818 	 action: 36.17   	 avg plaq: 0.8587  	 change: 0.0131  
Step: 4819 	 action: 37.48   	 avg plaq: 0.8536  	 change: -0.005125
Step: 4820 	 action: 36.99   	 avg plaq: 0.8555  	 change: 0.00192 
Step: 4821 	 action: 38.22   	 avg plaq: 0.8507  	 change: -0.004807
Step: 4822 	 action: 40.79   	 avg plaq: 0.8407  	 change: -0.01006
Step: 4823 	 action: 40.5    	 avg plaq: 0.8418  	 change: 0.001151
Step: 4824 	 action: 37.82   	 avg plaq: 0.8523  	 change: 0.01046 
Step: 4825 	 action: 38.14   	 avg plaq: 0.851   	 change: -0.001248
Step: 4826 	 action: 42.86   	 avg plaq: 0.8326  	 change: -0.01842
Step: 4827 	 action: 38.41   	 avg plaq: 0.85    	 change: 0.01735 
Step: 4828 	 action: 39.81   	 avg plaq: 0.8

Step: 4936 	 action: 36.47   	 avg plaq: 0.8575  	 change: -0.005895
Step: 4937 	 action: 32.13   	 avg plaq: 0.8745  	 change: 0.01698 
Step: 4938 	 action: 32.95   	 avg plaq: 0.8713  	 change: -0.003201
Step: 4939 	 action: 30.38   	 avg plaq: 0.8813  	 change: 0.01003 
Step: 4940 	 action: 32.17   	 avg plaq: 0.8743  	 change: -0.007005
Step: 4941 	 action: 34.43   	 avg plaq: 0.8655  	 change: -0.008826
Step: 4942 	 action: 33.1    	 avg plaq: 0.8707  	 change: 0.005214
Step: 4943 	 action: 31.95   	 avg plaq: 0.8752  	 change: 0.0045  
Step: 4944 	 action: 35.74   	 avg plaq: 0.8604  	 change: -0.0148 
Step: 4945 	 action: 35.03   	 avg plaq: 0.8632  	 change: 0.00275 
Step: 4946 	 action: 34.12   	 avg plaq: 0.8667  	 change: 0.003545
Step: 4947 	 action: 31.97   	 avg plaq: 0.8751  	 change: 0.008433
Step: 4948 	 action: 34.88   	 avg plaq: 0.8637  	 change: -0.01139
Step: 4949 	 action: 31.74   	 avg plaq: 0.876   	 change: 0.01228 
Step: 4950 	 action: 34.96   	 avg plaq: 0.8

In [190]:
np.mean(avg_plaq_arr[:100])

0.8640647

In [191]:
np.mean(avg_plaq_arr)

0.8630104

In [188]:
exact_plaquette_average(beta)

0.8635226110245504

In [ ]:
avg_plaq_arr = [0]
p = k - j
for k in range(j, 40000):
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[p-1]
    avg_plaq_arr.append(avg_plaq)
    print(f"Step: {k:<5g}: avg plaq: {avg_plaq:>12.4g} change: {change:12.4g}")
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            _ = u1_lattice._update_link(site, d)

In [ ]:
num_acceptances = 0
measure_steps = 10000
avg_plq = np.zeros(measure_steps)
for step in range(measure_steps):
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            num_acceptances += u1_lattice._update_link(site, d)
    avg_plq[step] = u1_lattice._average_plaquette()

In [ ]:
u1_lattice._total_action()

In [ ]:
u1_lattice._average_plaquette()

#### Run L2HMC for $U(1)$ gauge model

In [229]:
time_size, space_size, dim, beta, num_samples = (8, 8, 2, 2., 4)
conv_net = True
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples, rand=False)
if conv_net:
    u1_samples_tensor = tf.convert_to_tensor(u1_lattice.samples, 
                                             dtype=tf.float32)
else:
    flat_samples = [sample.flatten() for sample in u1_lattice.samples]
    u1_samples_tensor = tf.convert_to_tensor(np.stack(flat_samples), 
                                             dtype=tf.float32)

# Construct dynamics object
n_steps, eps = (10, 0.1) # number of leapfrog steps, initial step size
u1_energy_fn = u1_lattice.get_energy_function(u1_samples_tensor)
u1_dynamics = l2hmc.GaugeDynamics(u1_lattice, n_steps=n_steps, eps=eps,
                                  minus_loglikelihood_fn=u1_energy_fn, 
                                  conv_net=conv_net, test_HMC=True)

In [230]:
################  tests  #############################
#u1_lattice._total_action()
#u1_lattice._average_plaquette()
#u1_lattice.total_action(u1_samples)
#u1_lattice.total_action()
#u1_lattice.average_plaquette(u1_samples)
#u1_lattice.average_plaquette()
#---------------------------------------------
#_momentum = tf.random_normal(tf.shape(u1_samples))
#_potential = np.array(u1_dynamics.potential(u1_samples_tensor))
#_kinetic = u1_dynamics.kinetic(_momentum)
#_grad_potential = u1_dynamics.grad_potential(u1_samples_tensor)
#print(_potential); print('\n')
#print(_kinetic); print('\n')
#print(_grad_potential)
#print(_kinetic.numpy()); print('\n')
#print(_hamiltonian.numpy()); print('\n')
#print(_grad_potential[0][:10])
#---------------------------------------------
#site = u1_lattice.get_random_site()
#u = np.random.randint(u1_lattice.dim)
#v = np.random.randint(u1_lattice.dim)
#plaq = u1_lattice.plaquette_operator(site, u, v)
#---------------------------------------------
#u1_lattice.total_action(u1_samples_tensor)
#u1_lattice.average_plaquette(u1_samples_tensor)
#transition_out = u1_dynamics.apply_transition(u1_samples_tensor)
#x_post, p_post, accept_prob, x_out = transition_out
#loss, x_out, x_accept_prob = l2hmc.compute_loss(u1_dynamics, u1_samples_tensor)
#x_accept_prob
###############################################

In [231]:
# Create new log_dir with new run number
root_dir = '../../U1_logs/'
if not os.path.exists(root_dir):
    os.makedirs(root_dir)
log_dirs = os.listdir(root_dir)
try:
    run_nums = [int(i.split('_')[-1]) for i in log_dirs if i.startswith('run')]
    run_num = max(run_nums) + 1
except:
    run_num = 1
log_dir = f'../../U1_logs/run_{run_num}'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
print(f'{log_dir}')

../../U1_logs/run_157


In [232]:
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)
train_iters = 500
record_loss_every = 50
save_steps = 50 

learning_rate = tf.train.exponential_decay(1e-3, global_step, 50,
                                           0.96, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
summary_writer = tf.contrib.summary.create_file_writer(log_dir)
loss_fn = l2hmc.compute_loss
print(u1_plaq_exact(beta))

0.6977746579640083


In [ ]:
#################    Run L2HMC algorithm    ##################################
total_actions = []
average_plaquettes = []
t0 = time.time()
samples = u1_samples_tensor
for i in range(1, 500):
    t1 = time.time()
    loss, samples, accept_prob = train_one_iter(
        u1_dynamics,
        samples,
        optimizer,
        loss_fn=loss_fn,
        global_step=global_step
    )
    _total_actions = u1_lattice.total_action(samples)
    _avg_plaquettes = u1_lattice.average_plaquette(samples)
    total_actions.extend(_total_actions)
    average_plaquettes.extend(_avg_plaquettes)

    print("Iteration {}, loss: {:^.4f}, x_accept: {:^.4f},"
          " eps: {:^.4f}, avg_S: {:^.4f}, " #num_steps: {:^.4f}, "
          "avg_plaquette: {:^.4f}\n _avg_plaquettes: {}, "
          "\n _total_actions: {}\n" "time per training step: {:^4f}\n".format(
              i, loss.numpy(), accept_prob.numpy().mean(),
              u1_dynamics.eps.numpy(), #u1_dynamics.n_steps.numpy(), 
              np.mean(_total_actions), np.mean(_avg_plaquettes), 
              [i.numpy() for i in _avg_plaquettes],
              [i.numpy() for i in _total_actions], time.time() - t1
          ))

    if i % record_loss_every == 0:
        with summary_writer.as_default():
            with tf.contrib.summary.always_record_summaries():
                tf.contrib.summary.scalar("Training_loss", loss,
                                          step=global_step)

    if i % save_steps == 0:
        saved_path = checkpointer.save(file_prefix=os.path.join(log_dir,
                                                                "ckpt"))
        print(f"Saved checkpoint to: {saved_path}")
        avg_plaquette_file = os.path.join(log_dir, 'average_plaquettes.npy')
        np.save(avg_plaquette_file, np.array(average_plaquettes))
        total_actions_file = os.path.join(log_dir, 'total_actions.npy')
        np.save(total_actions_file, np.array(total_actions))

Iteration 1, loss: -666.4348, x_accept: 0.7688, eps: 0.1010, avg_S: 9.9660, avg_plaquette: 0.9221
 _avg_plaquettes: [1.0, 0.88892895, 0.89597166, 0.9036616], 
 _total_actions: [0.0, 14.217089, 13.315614, 12.331318]
time per training step: 98.576544

Iteration 2, loss: -873.1044, x_accept: 0.9031, eps: 0.1020, avg_S: 15.8926, avg_plaquette: 0.8758
 _avg_plaquettes: [0.91109216, 0.88892895, 0.8156735, 0.8876616], 
 _total_actions: [11.380186, 14.217089, 23.593775, 14.37931]
time per training step: 82.905575

Iteration 3, loss: -899.7542, x_accept: 0.9227, eps: 0.1030, avg_S: 20.0474, avg_plaquette: 0.8434
 _avg_plaquettes: [0.8536504, 0.8209834, 0.83511627, 0.86376876], 
 _total_actions: [18.73275, 22.91413, 21.10512, 17.437574]
time per training step: 83.630525

Iteration 4, loss: -863.1532, x_accept: 0.9074, eps: 0.1040, avg_S: 24.1948, avg_plaquette: 0.8110
 _avg_plaquettes: [0.8332598, 0.8126077, 0.82319075, 0.7748535], 
 _total_actions: [21.342756, 23.986235, 22.631596, 28.818739]
t

### Gradients check

Want to test that the value of the gradient ($\partial_x U(x)$) is the same when  
calculated using either:
1. Tensorflow's automatic differentiation (`tf.gradients`), or
2. The analytic expression (Eq. 17) from [reference](https://arxiv.org/pdf/hep-lat/9809160.pdf):
\begin{equation}
\partial_x U(x) = \beta \sum_{\nu \neq \mu} \sin \theta_{\mu\nu}(x - \hat\nu) - \sin \theta_{\mu\nu}(x)
\end{equation}

In [210]:
# fake data used to compare values of the gradient of the potential function
samples_ = tf.convert_to_tensor(
    [np.arange(128).reshape(u1_lattice.links.shape) for _ in range(4)],
    dtype=tf.float32
)

In [218]:
# using tensorflow's automatic differentiation 
tf_grad = u1_dynamics.grad_potential(samples) 
# 276 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

276 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [219]:
# using the analytic expression
# (Eq. 17 from https://arxiv.org/pdf/hep-lat/9809160.pdf)
exact_grad = u1_lattice.grad_action(samples)
# 503 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

503 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [217]:
print('exact grad:')
print(exact_grad[0].reshape(u1_lattice.links.shape)[:4, :4, 0])
print('\n')
print('tf grad:')
print(tf_grad[0].numpy().reshape(u1_lattice.links.shape)[:4, :4, 0])

exact grad:
[[-0.97712278  3.26007509 -1.55236864 -3.12323356]
 [-3.12247324 -0.2006627  -1.83291686 -0.71278107]
 [ 0.98088837  0.01663715 -1.67887139  0.48193204]
 [ 4.18461895 -1.57589948 -2.16718554  0.24280304]]


tf grad:
[[-0.9771228   3.260075   -1.5523686  -3.1232338 ]
 [-3.1224732  -0.20066273 -1.8329167  -0.7127812 ]
 [ 0.98088837  0.01663721 -1.6788715   0.48193204]
 [ 4.184619   -1.5758996  -2.1671855   0.24280298]]


In [220]:
print('exact grad:')
print(exact_grad[0].reshape(u1_lattice.links.shape)[:4, :4, 1])
print('\n')
print('tf grad:')
print(tf_grad[0].numpy().reshape(u1_lattice.links.shape)[:4, :4, 1])

exact grad:
[[-0.19238535 -2.73329401  0.84687883  3.08645296]
 [ 0.24397749  3.70471263  3.98526478  1.57481074]
 [-2.52266145 -2.73996425 -2.89400959 -4.0887208 ]
 [-0.34991017  1.24262476  1.73093772  1.97006512]]


tf grad:
[[-0.19238311 -2.7332976   0.84688044  3.086453  ]
 [ 0.2439785   3.7047162   3.9852643   1.5748118 ]
 [-2.5226645  -2.7399645  -2.8940098  -4.088723  ]
 [-0.349913    1.2426238   1.7309377   1.9700668 ]]


## OLD

### GMM Model (test L2HMC)

In [121]:
sigmas, distribution = gen_ring(1., var=0.02, nb_mixtures=4)

gmm_potential = distribution.get_energy_function()
gmm_dynamics = _l2hmc.Dynamics(x_dim=2, minus_loglikelihood_fn=gmm_potential,
                               n_steps=25, eps=0.1)

samples = distribution.get_samples(200)

_position = tf.convert_to_tensor(samples, dtype=tf.float32)
_momentum = tf.random_normal(tf.shape(_position))
_hamiltonian = gmm_dynamics.hamiltonian(_position, _momentum)

grad_pot = gmm_dynamics.grad_potential(_position, _momentum)

grad_pot.shape

gmm_dynamics.masks[0]

scale, translation, transformed = gmm_dynamics.position_fn([_position, _momentum, gmm_dynamics._get_time(0)])

scale.shape

_position.shape

gmm_dynamics.masks[0].shape

gmm_dynamics.masks[0] * _position

### Construct GaugeLattice with SU(3) gauge group

In [ ]:
time_size = 2
space_size = 4
dim = 4
beta = 1.
link_type = 'SU3' 
batch_size = 3
gauge_lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = gauge_lattice.get_links_samples(batch_size, link_type=link_type)

In [ ]:
gauge_energy_fn = gauge_lattice.get_energy_function()
gauge_dynamics = l2hmc.GaugeDynamics(gauge_lattice, 
                                     minus_loglikelihood_fn=gauge_energy_fn, 
                                     batch_size=3, n_steps=5, eps=0.1)

In [ ]:
gauge_lattice.links.shape

In [ ]:
potential_arr = gauge_dynamics.potential(links_samples, batch_size)

[i.numpy() for i in potential_arr]

In [ ]:
_momentum = tf.random_normal(tf.shape(links_samples))

In [ ]:
gauge_dynamics.kinetic(_momentum).numpy()

In [ ]:
_x = links_samples
#_momentum = tf.random_normal(tf.shape(_x))
_hamiltonian = gauge_dynamics.hamiltonian(_x, _momentum)
_hamiltonian

### Construct IsingLattice

In [ ]:
ising_batch_size = 10
ising_lattice = IsingLattice(3, 4)
ising_samples = [ising_lattice._randomize() for _ in range(ising_batch_size)]

In [ ]:
ising_energy_fn = ising_lattice.get_energy_function()
ising_dynamics = l2hmc.LatticeDynamics(ising_lattice, 
                                       minus_loglikelihood_fn=ising_energy_fn,
                                       batch_size=ising_batch_size, 
                                       n_steps=10, eps=0.1)
#dynamics = l2hmc.LDynamics(latt.sites.shape, minus_loglikelihood_fn=energy_fn, n_steps=10, eps=0.1)

In [ ]:
ising_dynamics.potential(samples, batch_size)

In [ ]:
_iposition = ising_samples
_imomentum = tf.random_normal(tf.shape(_iposition))
_ihamiltonian = dynamics.hamiltonian(_iposition, _imomentum)
_ihamiltonian

In [ ]:
_isample = _iposition[0].reshape(ising_lattice.num_sites)
#dynamics.grad_potential(np.array(_position).reshape(-1, lattice.num_sites))

In [ ]:
grad_pot = dynamics.grad_potential(ising_samples)

In [ ]:
ising_jacobian = jacobian(dynamics.potential, ising_samples)

In [ ]:
grad_fn = tfe.gradients_function(lattice._calc_energy, params=[0])

In [ ]:
_jacobian = jacobian(dynamics.potential, _position)

In [ ]:
lattice.calc_energy(_position, batch_size)

In [ ]:
#dynamics.position_fn(momentum, latt.sites.flatten()[:], dynamics)
#dynamics._forward_lf(latt.sites.flatten()[:], momentum, 0)
dynamics._forward_lf(np.array(_position).reshape(-1, lattice.num_sites),
                     np.array(_momentum).reshape(-1, lattice.num_sites), 1)